Santiago Rodriguez Avila

Juan Pablo Reyes Fajardo

# <p style="text-align: center;">Tarea 3</p>
## <p style="text-align: center;">(Montecarlo, SARSA, Q-Learning)
</p>

Para esta tarea se quiere estudiar el comportamiento de los algoritmos de aprendizaje por refuerzo de: Montecarlo, SARSA y Q-learning. Se observarán características, comportamiento y composición de estos. Se hablará de políticas on y off (que el agente sigue la misma política a evaluar o una diferente), todo esto partiendo del escenario de la tarea anterior, el juego de escaleras y serpientes.

## Inicialización del MDP:

El MDP formuldado en el módulo de Python describe un  juego de escaleras y serpientes. Este permite:

* Dada una acción en un estado, retornar la recompensa y el estado resultante en esa acción. Considera el caso especial del estado terminal.


* Ejecutar una política arbitraria.

Importación de módulos:

In [1]:
from types import MethodType
from escalerasyserpientes import EscalerasSerpientes
import copy
import numpy as np
import random
import pandas as pd

Inicialización del MDP con:
* Casillas de victoria en (80,100)
* Casillas de pérdida en (23,37,45,67,89)
* Recompensa por paso de 0


In [2]:
es=EscalerasSerpientes([80,100],[23,37,45,67,89],0)

Fucnion para visualizar estados del MDP, la política y los valores de Q:

In [3]:
def imprimir_MDP():
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(pd.DataFrame({"Política: ": list(es.policy.values()), "Q: ":list(es.Q.values())},index=es.states).round(decimals=3))

Valores iniciales:

In [4]:
imprimir_MDP()

,Política:,Q:
1,Ad,"{'Ad': 0, 'At': 0}"
2,Ad,"{'Ad': 0, 'At': 0}"
3,Ad,"{'Ad': 0, 'At': 0}"
4,Ad,"{'Ad': 0, 'At': 0}"
5,Ad,"{'Ad': 0, 'At': 0}"
6,Ad,"{'Ad': 0, 'At': 0}"
7,Ad,"{'Ad': 0, 'At': 0}"
8,Ad,"{'Ad': 0, 'At': 0}"
9,Ad,"{'Ad': 0, 'At': 0}"
10,Ad,"{'Ad': 0, 'At': 0}"


### Pruebas:

La estructura de retorno del MDP es la siguiente: Casilla de llegada, valor de la recompensa obtenida en el paso,  acción tomada, movimiento realizados (valor del dado)y la variable que muestra si el estado al que se llegó es un estado terminal. En ese orden de ideas, las pruebas serían:

Probar función de pasos para movimientos determinísticos del agente para verificar victorias, perdidas, rebotes, escaleras y serpientes.

In [5]:
state_prueba=97
action_prueba=5
pruebas=[['Caso serpiente',96,2],
         ['Caso rebote inicio',2,-6],
         ['Caso rebote fin y serpiente',97,5],
         ['Caso victoria',94,6],
         ['Caso terminal',100,0],
         ['Caso fracaso',35,2],
         ['Caso terminal',37,0],
         ['Caso escalera',49,1]]
for prueba in pruebas:
    state_, reward, action, real_action, done=es.step(prueba[1], prueba[2], random=False)

    display(f'{prueba[0]}: Comienza en: {prueba[1]}, se mueve: {prueba[2]}, termina en {state_} y obtiene recompensa {reward}')

'Caso serpiente: Comienza en: 96, se mueve: 2, termina en 28 y obtiene recompensa 0'

'Caso rebote inicio: Comienza en: 2, se mueve: -6, termina en 6 y obtiene recompensa 0'

'Caso rebote fin y serpiente: Comienza en: 97, se mueve: 5, termina en 28 y obtiene recompensa 0'

'Caso victoria: Comienza en: 94, se mueve: 6, termina en 100 y obtiene recompensa 0'

'Caso terminal: Comienza en: 100, se mueve: 0, termina en Azul y obtiene recompensa 1.0'

'Caso fracaso: Comienza en: 35, se mueve: 2, termina en 37 y obtiene recompensa 0'

'Caso terminal: Comienza en: 37, se mueve: 0, termina en Rojo y obtiene recompensa -1.0'

'Caso escalera: Comienza en: 49, se mueve: 1, termina en 91 y obtiene recompensa 0'

Probar función de pasos para acciones del agente para verificar el movimiento aleatorio dado una acción "adelante" o "atrás":

In [6]:
state_prueba=98
accion_prueba='Ad'
for i in range(3**2):
    state_, reward, action, real_action, done=es.step(state_prueba, accion_prueba, random=True)

    display(f'Comienza en: {state_prueba}, acción: {accion_prueba}, termina en {state_} y obtiene recompensa {reward}')

'Comienza en: 98, acción: Ad, termina en 99 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 99 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 97 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 99 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 100 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 28 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 97 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 99 y obtiene recompensa 0'

'Comienza en: 98, acción: Ad, termina en 99 y obtiene recompensa 0'

## Generadores de episodios:

### Generador de episodios regular:

Juega el tablero usando la poítica del MDP y termina hasta que gana o pierde.

Retorna el número de pasos y el estado de la casilla final.

In [7]:
def episodio():
    state=1
    steps=0
    while True:
        state_, _,_, _, done=es.step(state, es.policy[state], random=True)
        state=state_
        steps+=1
        if done:
            return state_,steps


### Generador de episodios MonteCarlo

Juega el tablero usando la política en el argumento y termina hasta que gana o pierde.

Retorna listas con los estados, recompensas, acciones intentadas, movimientos, terminalidad, probabilidades de la política usada y número de pasos.

In [8]:
def episodio_MC(policy):
    state=1
    steps=0
    state_s=[1]
    rewards=[]
    tried_actions=[]
    real_actions=[]
    dones=[]
    probs=[]
    while True:
        pol,prob=policy(state)

        state_, reward,try_action, real_action, done=es.step(state, pol, random=True)
        state_s.append(state_)
        rewards.append(reward)
        real_actions.append(real_action)
        dones.append(done)
        probs.append(prob)
        tried_actions.append(try_action)
        
        state=state_
        
        steps+=1
        
        if done:
            return state_s,rewards,tried_actions,real_actions,dones,probs,steps

### Múltiples episodios:

In [9]:
def n_episodios(n):
    counter=0
    stepss=[]
    for i in range(n):
        state_,steps=episodio()
        if state_=='Azul':
            counter+=1
            stepss.append(steps)
    print(f'Victorias: {100*counter/n}% Pasos promedio: {sum(stepss)/len(stepss)}')

In [10]:
n_episodios(1000)

Victorias: 10.0% Pasos promedio: 19.19


## Algoritmo MonteCarlo Off-Policy

### Política de comportamiento (MonteCarlo):

Retorna una acción aleatoria dentro de las posibilidades de un estado y la probabilidad de la acción en el estado.

In [11]:
def random_policy(state):
    return random.choice(list(es.Q[state].keys())),1/2

### Control con MonteCarlo Off-Policy para estimar $\pi^*:$

In [12]:
def MC_offpolicy(es,gamma):
    #Genera el episodio
    state_s,rewards,tried_actions,real_actions,dones,probs,steps=episodio_MC(random_policy)
    G=0
    W=1
    for i in reversed(range(len(state_s[0:-2]))):
        G=gamma*G+rewards[i+1]
        es.C[state_s[i]][tried_actions[i]]=es.C[state_s[i]][tried_actions[i]]+W
        es.Q[state_s[i]][tried_actions[i]]=es.Q[state_s[i]][tried_actions[i]]+(W/(es.C[state_s[i]][tried_actions[i]]))*(G-es.Q[state_s[i]][tried_actions[i]])
        es.policy[state_s[i]]=es.key_max(es.Q[state_s[i]])[0]

        if tried_actions[i]!=es.policy[state_s[i]]:
            break

        W=W*(1/probs[i])

### Inicialización y aprendizaje de política:

In [13]:
es.Q=es.init_Q(0,0)
es.C=es.init_Q(0,0)
es.policy = dict.fromkeys(es.states, 'Ad')
for i in range(0,100000):
    MC_offpolicy(es,0.9)

In [14]:
imprimir_MDP()

,Política:,Q:
1,Ad,"{'Ad': 0.24269385543185829, 'At': 0.1803546972..."
2,At,"{'Ad': 0.20490558331425757, 'At': 0.2054634219..."
3,Ad,"{'Ad': 0.14913758034669675, 'At': 0.0359183211..."
4,Ad,"{'Ad': 0.2294330334481121, 'At': 0.19457951548..."
5,At,"{'Ad': 0.1400302297576854, 'At': 0.18387388845..."
6,At,"{'Ad': 0.15353334997730902, 'At': 0.1553569288..."
7,Ad,"{'Ad': 0.2635196743804948, 'At': 0.18998774412..."
8,Ad,"{'Ad': 0, 'At': 0}"
9,Ad,"{'Ad': 0.35259535063077657, 'At': 0.1594030976..."
10,Ad,"{'Ad': 0.54772293289163, 'At': 0.1958564079347..."


In [15]:
n_episodios(1000)

Victorias: 95.0% Pasos promedio: 43.68421052631579


## Algoritmo SARSA

En esta parte de la tarea se va a analizar y desarrollar el algoritmo SARSA. Como se pudo ver en clase, este es un algoritmo *on-policy*, esto quiere decir que el algoritmo aprende directamente de la política actual del agente. Otra cosa que vale la pena nombrar es que este algoritmo utiliza la **función Q** para actualizar el valor del par estado, acción a evaluar. La norma de actualización de SARSA es la siguiente:

$$Q(s,a) \leftarrow Q(s,a)+\alpha*(R+\gamma*Q(s',a') - Q(s,a))$$

Donde $s'$ es el estado siguiente al que se llega luego de elegir $a$ de forma $\epsilon-greedy$ , y $a'$ es la acción $\epsilon-greedy$ elegida desde el estado $s'$. Finalmente, se repite esta regla de actualización actualizando los valores de forma: $s\leftarrow s'$ y $a\leftarrow a'$ hasta que se llegue a un estado terminal y ahí se termina el episodio.



### Política $\epsilon - greedy$:

Retorna una acción aleatoria dentro de las posibilidades de un estado y la probabilidad de la acción en el estado.

In [91]:
def eps_greedy(state,epsilon=0.1):
    rand = np.random.uniform()
    if rand>epsilon:
        return es.key_max(es.Q[state])[0],1-epsilon
    else:
        return es.key_min(es.Q[state])[0],epsilon

### Estimación de Q con SARSA

In [153]:
def SARSA(es,gamma,alpha):
    done=False
    state=1
    action,_=eps_greedy(state)
    while not done:
        state_, reward, try_action, real_action, done=es.step(state, action, random=True)
        if done:
            Q_=0
        else:
            action_,_=eps_greedy(state_)
            Q_=es.Q[state_][action_]
            
        es.Q[state][action]+=alpha*(reward+gamma*Q_-es.Q[state][action])
        state=state_
        action=action_     

### Inicialización y aprendizaje de Q:

In [154]:
es.Q=es.init_Q(0,0)
es.policy = dict.fromkeys(es.states, 'Ad')
for i in range(0,100000):
    SARSA(es,0.9,0.1)

### Política greedy respecto a Q

Luego de correr la implementación del algoritmo SARSA, la política que se va a utilizar va a ser el valor máximo de la función Q de cada par estado acción por estado. Por ejemplo, la política en el estado 1 va a ser el valor máximo de la **función Q** entre *q(1,"adelante")* y *q(1,"atras")*. 

Teniendo esto en cuenta, la política del algoritmo SARSA es la siguiente:

In [156]:
for state in es.states:
    es.policy[state],_=eps_greedy(state,0)

Luego de obtener la política, se procede a hacer una prueba de esta con un total de 1000 episodios. La idea es obtener el promedio de pasos obtenido por cada episodio y el porcentaje de victoria (donde el estado terminal sea "Azul") de los 1000 episodios.

In [160]:
n_episodios(1000)

Victorias: 89.7% Pasos promedio: 39.06577480490524


## Algoritmo Q-Learning

### Estimación de Q con Q-Learning

In [179]:
def Q_Learning(es,gamma,alpha):
    done=False
    state=1
    
    while not done:
        action,_=eps_greedy(state,0.4)
        state_, reward, try_action, real_action, done=es.step(state, action, random=True)
        if done:
            Q_=0
        else:
            Q_=es.key_max(es.Q[state_])[1]
        es.Q[state][action]+=alpha*(reward+gamma*Q_-es.Q[state][action])
        state=state_


### Inicialización y aprendizaje de Q:

In [227]:
es.Q=es.init_Q(0,0)
es.policy = dict.fromkeys(es.states, 'Ad')
for i in range(0,100000):
    Q_Learning(es,0.9,0.25)

### Política greedy respecto a Q

In [228]:
for state in es.states:
    es.policy[state],_=eps_greedy(state,0)

In [229]:
n_episodios(1000)

Victorias: 95.7% Pasos promedio: 44.75757575757576


In [223]:
imprimir_MDP()

,Política:,Q:
1,Ad,"{'Ad': 0.1425626926756141, 'At': 0.13028514721..."
2,Ad,"{'Ad': 0.16462036172914177, 'At': 0.1339504905..."
3,Ad,"{'Ad': 0.14567055706632276, 'At': 0.1367602423..."
4,Ad,"{'Ad': 0.13681836700219174, 'At': 0.1334763954..."
5,Ad,"{'Ad': 0.14119423118734742, 'At': 0.1295219623..."
6,Ad,"{'Ad': 0.14797687337336873, 'At': 0.1326969173..."
7,Ad,"{'Ad': 0.1768646002859781, 'At': 0.13187352073..."
8,Ad,"{'Ad': 0, 'At': 0}"
9,Ad,"{'Ad': 0.1661059495192842, 'At': 0.13128264022..."
10,Ad,"{'Ad': 0.16107886654974654, 'At': 0.1527150414..."
